<a href="https://colab.research.google.com/github/lbal-biomat/quality/blob/main/log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El paper en el que me guío es:

Nicholls, S. M., Quick, J. C., Tang, S., & Loman, N. J. (2019). Ultra-deep, long-read nanopore sequencing of mock microbial community standards. Gigascience, 8(5), giz043. https://academic.oup.com/gigascience/article/8/5/giz043/5486468

Los programas de polishing racon y homopolish/helen usan información de calidad por base. En el paper que elegí usan racon.
La idea es seguir los pasos del paper y ver que todo es reproducible, luego ensamblar con flye los últimos datos que publicaron (R10.3) y hacer el polish con distintos pipelines, todos usando la calidad en algún punto

- Flye + racon x1
- Flye + racon x1 + medaka
- Flye + racon x1 + homopolish
- Flye + racon x1 + medaka + homopolish
- Flye + racon x2
- Flye + racon x2 + medaka
- Flye + racon x2 + homopolish
- Flye + racon x2 + medaka + homopolish
- Flye + racon x4
- Flye + racon x4 + homopolish
- Flye + MarginPolish + helen
- Flye + MarginPolish + homopolish
- Flye + MarginPolish + helen + homopolish

 <br/>

**Instalación de programas**

Instalé las versiones correspondientes al paper con conda:

```
conda create --prefix ./env_paper python=3.6 pilon=1.23 racon=1.3.2 minimap2=2.14 kraken2 checkm-genome quast=5.0.2
```
